In [2]:
# Importing all libraries

import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Functions import *

In [3]:
#============== System parameters
'''
Defining Parameters
    Nt = Number of transmit antennas (SISO = 1)
    K = Number of Users
    BW = Bandwidth
    NF = Noise Floor
    ALPHA = PLE for indirect link (IRS-1)
    BETA = PLE for indirect link (IRS-2)
    ZETA = PLE for direct link 
    wn = efficiency of transmit power amplifier
    Pb = Power dissipated by BS
    Pu = Power dissipated by User
    loc_U = User positions
    m = Nakagami Shape parameter (m > 0, m = 1 corresponds to Rayleigh fading)
    omega = 1.0  Nakagami Spread parameter (omega > 0)
    Ns_max = Total number of IRS elements in a single IRS

'''
# Initialize constants
Nt = 1
f = 2.4e9
c = 3e8
Lambda = c / f
halfLambda = 0.5 * Lambda
quarterLambda = 0.25 * Lambda
BW = 20e6
NF = 10
noise = -174 + 10*np.log10(BW) + NF

# Network Parameters
BASE_STATION_POSITION = (0, 0, 35)
dThreshold = 50  # Replace GRID_RADIUS with appropriate value / 40
RATE_THRESHOLD = 25

# IRS Position
IRS_position = (50, 86.60, 10)

# IRS elements and dimensions
L = 4 # Number of faces for the IRS (change this for different polygons)
M = 360 // L
nIRSrow = M // 6
nIRScol = M // nIRSrow
edge_length = 5  # Length of each edge of the IRS in meters
z_height = edge_length  # Fixed height equal to edge_length

# User location function parameters
near_distance = 40
far_distance = 70

In [4]:
# Example usage
near_distance = 40
far_distance = 70
power_allocation_factor = 0.3
L = 5 # Example value for L

radius = 100
USER_POS, K = generate_user_positions(L, IRS_position, near_distance, far_distance)
locU = USER_POS 
xt, yt, zt = BASE_STATION_POSITION  # Coordinates of BS
locT = np.array([BASE_STATION_POSITION])  # Position of BS

coordinates, vertices, face_midpoints = generate_all_irs_coordinates(IRS_position, L, nIRSrow, nIRScol, edge_length, z_height)
dTU, dSU, dTS, ci = calculate_distances_3D(USER_POS, locT, face_midpoints, IRS_position, L)

# Generate circle and sectors data for plotting
x_circle, y_circle, z_circle, sector_lines = generate_circle_and_sectors_3D(IRS_position, radius, L)

# # Compute path gains
GTU, GSU, GTS = compute_path_gains(dTU, dSU, dTS, M)

hTU, hSU, hTS = compute_channels_NOMA(GTU, GSU, GTS, K, L, M, K_factor=10)

# print("Channel shape between users and BS:", np.shape(hTU))
# print("Channel shape between users and closest IRS faces:", np.shape(hSU))
# print("Channel shape between BS and all IRS faces:", np.shape(hTS))
# print()
# # print("Channels between users and BS:", (hTU))
# print("Channels between users and closest IRS faces:", (hSU))
# # print("Channels between BS and all IRS faces:", (hTS))
# print("Distance shape between users and BS:", np.shape(dTU))
# print("Distance shape between users and closest IRS faces:", np.shape(dSU))
# print("Distance shape between BS and all IRS faces:", np.shape(dTS))
# print()
# print("Distances between users and BS:\n", dTU)
# print("Distances between users and closest IRS faces:\n", dSU)
# print("Distances between BS and all IRS faces:\n", dTS)

In [5]:
# Plotting the coordinates in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(coordinates[:, 0], coordinates[:, 1], coordinates[:, 2], label='IRS Coordinates')

# Plot the edges of the faces (only top and bottom edges)
plot_edges(ax, vertices, nIRSrow, z_height)

# Setting labels
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate')
plt.title(f'3D Plot of IRS Coordinates for L = {L}, Edge Length = {edge_length} meters, Height = {z_height} meters')
plt.legend()
plt.show()

[0 0 1 1 2 2 3 3 4 4]


In [6]:
# Plotting in 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot IRS coordinates
ax.scatter(coordinates[:, 0], coordinates[:, 1], coordinates[:, 2], color='red', label='IRS Coordinates')

# Plot the edges of the faces (only top and bottom edges)
plot_edges(ax, vertices, nIRSrow, z_height)

# Plot wireless network environment
ax.scatter3D(USER_POS[:, 0], USER_POS[:, 1], USER_POS[:, 2], color='green', marker='s', label='Users')
ax.scatter(xt, yt, zt, color='blue', marker='*', label='Base Station')

# Plot the circle in the XY plane
ax.plot3D(x_circle, y_circle, z_circle, 'b--', linewidth=1)

# Plot the sector lines
for line in sector_lines:
    (x_start, y_start, z_start), (x_end, y_end, z_end) = line
    ax.plot3D([x_start, x_end], [y_start, y_end], [z_start, z_end], 'g-', linewidth=1)

# Setting labels and title
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate')
ax.set_title('Wireless Network Environment')
ax.legend()
plt.show()

print(ci)

# BD RIS IMPLEMENTATION (WITHOUT NOMA)

In [7]:
# Initialize constants
Nt = 1
f = 2.4e9
c = 3e8
Lambda = c / f
halfLambda = 0.5 * Lambda
quarterLambda = 0.25 * Lambda
BW = 10e6
NF = 10
noise = -174 + 10 * np.log10(BW) + NF

# Constants for large scale fading 
Gt = 1
Gr = 1
eta_RIS = 2
eta_kl = 2

# Network Parameters
BASE_STATION_POSITION = (0, 0, 20)
dThreshold = 50  # Replace GRID_RADIUS with appropriate value / 40
RATE_THRESHOLD = 0.25

# IRS Position
IRS_position = (10, 10, 5)
# IRS_position = (70, 106.60, 0)

# IRS elements and dimensions
edge_length = 5 # Length of each edge of the IRS in meters
z_height = edge_length  # Fixed height equal to edge_length

# Generate Users and Number of IRS Faces
radius = 30  # Range for generation of users within IRS
xt, yt, zt = BASE_STATION_POSITION  # Coordinates of BS
locT = np.array([BASE_STATION_POSITION])  # Position of BS

# Values for NOMA
near_distance = 40
far_distance = 70
power_allocation_factor = 0.25

# Different values of L (number of IRS faces) to simulate
NUM_SIMULATIONS = 100
L_max = 6
L_values = np.array([i for i in range(2, L_max + 1)])  # Example values, adjust as needed
Pt = np.arange(0, 51, 1)  # in dBm

# Initialize an empty dictionary to store the results
results_dict_2 = {}

for L in L_values:
    average_spectral_efficiency_2 = []

    # Ensure L*M = 360
    M = 360 // L
    nIRSrow = M // 6
    nIRScol = M // nIRSrow

    # Generate user positions with respect to the IRS faces (L)
    USER_POS, K = generate_user_positions(L, IRS_position, near_distance, far_distance)
    locU = USER_POS 

    arr_sumrate_disc = np.zeros((NUM_SIMULATIONS, 1))
    arr_rate_disc = np.zeros((NUM_SIMULATIONS, K, 1))
    arr_SNR_disc = np.zeros((NUM_SIMULATIONS, K, 1))
    arr_lb_disc = np.zeros((NUM_SIMULATIONS, K, 1))

    # Generate IRS coordinates, vertices, and face midpoints for the current L
    coordinates, vertices, face_midpoints = generate_all_irs_coordinates(IRS_position, L, nIRSrow, nIRScol, edge_length, z_height)

    # Calculate distances for IRS and determine the closest face for each user
    dTU, dSU, dTS, closest_face_indices = calculate_distances_3D(USER_POS, locT, face_midpoints, IRS_position, L)

    # Generate large scale fading coefficients once
    alpha_kl = generate_large_scale_fading_new(dTS, dSU, L, K, closest_face_indices)

    # Take the maximum value in each row of alpha_kl to create a matrix of (K, 1)
    alpha_kl_new = np.max(alpha_kl, axis=1).reshape(K, 1)

    # Compute path gains
    GTU, GSU, GTS = compute_path_gains(dTU, dSU, dTS, M)

    # Initialize arrays to store results for each L value
    h_dk_3d = np.zeros((NUM_SIMULATIONS, K, 1), dtype=np.complex128)
    h_rk1_3d = np.zeros((NUM_SIMULATIONS, M, K), dtype=np.complex128)
    h_rk1_h_3d = np.zeros((NUM_SIMULATIONS, K, M), dtype=np.complex128)
    G1_3d = np.zeros((NUM_SIMULATIONS, L, M), dtype=np.complex128)

    # Perform simulations
    for i in range(NUM_SIMULATIONS):
        # Compute channels
        hTU, hSU, hTS = compute_channels_NOMA(GTU, GSU, GTS, K, L, M, K_factor=10)

        # Store results for each simulation
        h_dk_3d[i] = hTU  # BS to users
        h_rk1_3d[i] = hSU  # IRS to users
        h_rk1_h_3d[i] = np.conj(np.transpose(hSU))  # Transpose IRS to users
        G1_3d[i] = hTS  # BS to all IRS faces and their elements

    for T in Pt:
        # Converting the dBm power to watts for further use
        TRANSMIT_POWER_WATTS = dBm2pow(T)

        # Initialize sum-rates for this case
        spectral_efficiency = []

        # Perform simulations
        for s in range(NUM_SIMULATIONS):
            h_dk = h_dk_3d[s]  # Shape: (K, 1)
            h_rk = h_rk1_3d[s]  # Shape: (M, K)
            h_rk_transpose = h_rk1_h_3d[s]  # Shape: (K, M)
            G = G1_3d[s]  # Shape: (L, M)

            # Compute theta and product matrix
            theta = theta_matrix_ideal(1, h_dk*0, h_rk, G, K, M, L)
            results_array = prod_matrix(theta, h_rk_transpose, G, K, M, L, closest_face_indices)

            # Calculate sum rate
            arr_link_budget = np.abs((TRANSMIT_POWER_WATTS * alpha_kl_new)**0.5 * (h_dk*0 + results_array))
            arr_link_budget = pow2dBm((arr_link_budget)**2)
            arr_SNR_disc[s, :, :] = compute_SNR(arr_link_budget, noise)
            arr_rate_disc[s, :, :] = compute_rate(arr_SNR_disc[s, :, :])
            arr_sumrate_disc[s] = np.sum(arr_rate_disc[s, :, :])
            spectral_efficiency.append(arr_sumrate_disc[s]/K)

        # Calculate average sum rate across simulations for this Pt
        average_spectral_efficiency_2.append(np.mean(spectral_efficiency))

    # Store average sum rate for this L in the results dictionary
    results_dict_2[L] = average_spectral_efficiency_2

print('done')
print(L_values)

done
[2 3 4 5 6]


In [8]:
for L, average_spectral_efficiency_2 in results_dict_2.items():
    plt.plot(Pt, average_spectral_efficiency_2, label=f'L={L}')
    
plt.ylabel('Spectral Efficiency (bit/s/Hz)')
plt.xlabel('Transmit Power (dBm)')
plt.title('Spectral Efficiency Vs Transmit Power')
plt.legend()
plt.grid(True)
plt.show()

# BD-RIS IMPLEMENTATION (WITH NOMA)

In [9]:
# Initialize an empty dictionary to store the results
results_dict_1 = {}

for L in L_values:
    average_spectral_efficiency_1 = []

    # Ensure L*M = 360
    M = 360 // L
    nIRSrow = M // 6
    nIRScol = M // nIRSrow

    # Generate user positions with respect to the IRS faces (L)
    USER_POS, K = generate_user_positions(L, IRS_position, near_distance, far_distance)
    locU = USER_POS 

    arr_sumrate_disc = np.zeros((NUM_SIMULATIONS, 1))
    arr_rate_disc = np.zeros((NUM_SIMULATIONS, K, 1))
    outage_probability_disc = np.zeros((NUM_SIMULATIONS, K, 1))
    arr_SNR_disc = np.zeros((NUM_SIMULATIONS, K, 1))
    arr_lb_disc = np.zeros((NUM_SIMULATIONS, K, 1))

    # Generate IRS coordinates, vertices, and face midpoints for the current L
    coordinates, vertices, face_midpoints = generate_all_irs_coordinates(IRS_position, L, nIRSrow, nIRScol, edge_length, z_height)

    # Calculate distances for IRS and determine the closest face for each user
    dTU, dSU, dTS, closest_face_indices = calculate_distances_3D(USER_POS, locT, face_midpoints, IRS_position, L)

    # Generate large scale fading coefficients once
    alpha_kl = generate_large_scale_fading_new(dTS, dSU, L, K, closest_face_indices)

    # Take the maximum value in each row of alpha_kl to create a matrix of (K, 1)
    alpha_kl_new = np.max(alpha_kl, axis=1).reshape(K, 1)

    # Compute path gains
    GTU, GSU, GTS = compute_path_gains(dTU, dSU, dTS, M)

    # Initialize arrays to store results for each L value
    h_dk_3d = np.zeros((NUM_SIMULATIONS, K, 1), dtype=np.complex128)
    h_rk1_3d = np.zeros((NUM_SIMULATIONS, M, K), dtype=np.complex128)
    h_rk1_h_3d = np.zeros((NUM_SIMULATIONS, K, M), dtype=np.complex128)
    G1_3d = np.zeros((NUM_SIMULATIONS, L, M), dtype=np.complex128)

    # Perform simulations
    for i in range(NUM_SIMULATIONS):
        # Compute channels
        hTU, hSU, hTS = compute_channels_NOMA(GTU, GSU, GTS, K, L, M, K_factor=10)

        # Store results for each simulation
        h_dk_3d[i] = hTU  # BS to users
        h_rk1_3d[i] = hSU  # IRS to users
        h_rk1_h_3d[i] = np.conj(np.transpose(hSU))  # Transpose IRS to users
        G1_3d[i] = hTS  # BS to all IRS faces and their elements

    for T in Pt:
        # Converting the dBm power to watts for further use
        TRANSMIT_POWER_WATTS = dBm2pow(T)

        # Initialize sum-rates for this case
        spectral_efficiency = []

        # Perform simulations
        for s in range(NUM_SIMULATIONS):
            h_dk = h_dk_3d[s]  # Shape: (K, 1)
            h_rk = h_rk1_3d[s]  # Shape: (M, K)
            h_rk_transpose = h_rk1_h_3d[s]  # Shape: (K, M)
            G = G1_3d[s]  # Shape: (L, M)

            # Compute theta and product matrix
            theta = theta_matrix_ideal(1, h_dk*0, h_rk, G, K, M, L)
            results_array = prod_matrix(theta, h_rk_transpose, G, K, M, L, closest_face_indices)

            # Calculate sum rate
            arr_link_budget = np.abs((TRANSMIT_POWER_WATTS * alpha_kl_new)**0.5 * (h_dk*0 + results_array))
            arr_link_budget = pow2dBm((arr_link_budget)**2)
            arr_SNR_disc[s, :, :] = compute_SNR_NOMA(arr_link_budget, noise)
            arr_rate_disc[s, :, :] = compute_noma_rates(arr_SNR_disc[s, :, :], K, power_allocation_factor)
            arr_sumrate_disc[s] = np.sum(arr_rate_disc[s, :, :])
            spectral_efficiency.append(arr_sumrate_disc[s])

        # Calculate average sum rate across simulations for this Pt
        average_spectral_efficiency_1.append(np.mean(spectral_efficiency))

    # Store average sum rate for this L in the results dictionary
    results_dict_1[L] = average_spectral_efficiency_1

print('done')
print(L_values)

done
[2 3 4 5 6]


In [10]:
for L, average_spectral_efficiency_1 in results_dict_1.items():
    plt.plot(Pt, average_spectral_efficiency_1, label=f'L={L}')
    
plt.ylabel('Spectral Efficiency (bit/s/Hz)')
plt.xlabel('Transmit Power (dBm)')
plt.title('Spectral Efficiency Vs Transmit Power')
plt.legend()
plt.grid(True)
plt.show()